In [90]:
import os
import pandas as pd
import numpy as np
import geopandas as gp
import json
from sklearn.model_selection import train_test_split
from sklearn import svm

curr_dir = os.path.abspath('')

# Import Google Semantic Data for February 2015

In [70]:
# Load file and filter out all non-acitivty segments
with open('GoogleLocationHistory/Takeout/Location History/Semantic Location History/2015/2015_FEBRUARY.json') as f:
    data = json.load(f)
data = data['timelineObjects']
data = [x['activitySegment'] for x in data if 'activitySegment' in x]
print(data)

[{'startLocation': {'latitudeE7': 400105149, 'longitudeE7': -755851868, 'placeId': 'ChIJyeJij6jzxokRKWDVxqQm7_s', 'address': '502 Cork Cir\nWest Chester, PA 19380\nUSA', 'name': '502 Cork Cir', 'semanticType': 'TYPE_HOME', 'locationConfidence': 99.97496}, 'endLocation': {'latitudeE7': 399518684, 'longitudeE7': -756002490, 'placeId': 'ChIJBxuyYRXxxokR88T7gl6xwio', 'address': '700 S High St\nWest Chester, PA 19383\nUSA', 'name': 'West Chester University of Pennsylvania', 'locationConfidence': 68.86896}, 'duration': {'startTimestampMs': '1423354387792', 'endTimestampMs': '1423355400750'}, 'confidence': 'MEDIUM', 'activities': [{'activityType': 'IN_VEHICLE', 'probability': 0.0}, {'activityType': 'CYCLING', 'probability': 0.0}, {'activityType': 'FLYING', 'probability': 0.0}], 'waypointPath': {'waypoints': [{'latE7': 400102233, 'lngE7': -755856018}, {'latE7': 399519386, 'lngE7': -755993957}]}, 'editConfirmationStatus': 'NOT_CONFIRMED'}, {'startLocation': {}, 'endLocation': {}, 'duration': {'

In [77]:
df = pd.DataFrame(data)
df.duration = df.duration.apply(lambda x: int(x['endTimestampMs']) - df.duration.apply(lambda x: int(x['startTimestampMs'])))
df.activities = df.activities.apply(lambda x: x[0]['activityType'])

                                        startLocation  \
0   {'latitudeE7': 400105149, 'longitudeE7': -7558...   
1                                                  {}   
2                                                  {}   
3   {'latitudeE7': 400141710, 'longitudeE7': -7567...   
4   {'latitudeE7': 400104805, 'longitudeE7': -7558...   
..                                                ...   
70  {'latitudeE7': 399796847, 'longitudeE7': -7556...   
71  {'latitudeE7': 400105149, 'longitudeE7': -7558...   
72  {'latitudeE7': 399796847, 'longitudeE7': -7556...   
73                                                 {}   
74                                                 {}   

                                          endLocation    duration confidence  \
0   {'latitudeE7': 399518684, 'longitudeE7': -7560...     1012958     MEDIUM   
1                                                  {}     7857766     MEDIUM   
2                                                  {}    69462633     MEDIU

In [82]:
simple = df[['duration', 'distance', 'activities']]
simple = simple.dropna()
print(simple)

      duration  distance             activities
1      7857766    8020.0             IN_VEHICLE
2     69462633    7634.0             IN_VEHICLE
3     77602925    7838.0             IN_VEHICLE
4    129154457    3887.0                CYCLING
5    157653146     481.0  UNKNOWN_ACTIVITY_TYPE
..         ...       ...                    ...
66  1634039195    3893.0             IN_VEHICLE
68  1689398120   11236.0             IN_VEHICLE
70  1720240846    8442.0             IN_VEHICLE
73  1802436992   63299.0             IN_VEHICLE
74  1812549260   12788.0             IN_VEHICLE

[61 rows x 3 columns]


In [93]:
x_train, x_test, y_train, y_test = train_test_split(simple[['duration','distance']], simple['activities'], test_size = 0.2)

model = svm.SVC()
model.fit(x_train, y_train)

percentage = model.score(x_test, y_test)
print(f"Simple Classification Accuracy = {percentage * 100} %")

Simple Classification Accuracy = 69.23076923076923 %
